<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/128_Docker_Container_Creation_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## 🧠 Why Containers Are Great for AI Agents

1. **Isolation**

   * Keeps your dependencies separate from other projects.
   * You can have one container using Claude, another using OpenAI, another for experiments.

2. **Reproducibility**

   * If you get an agent working today, you can still rebuild the same environment 6 months later.

3. **Safety with Secrets**

   * Your `.env` keys (Claude, OpenAI, etc.) never get baked into the image.
   * You inject them at runtime, so they’re safe and private.

4. **Portability**

   * The same container works on your Mac, a cloud server, or even someone else’s laptop — no “but it worked on my machine” issues.



# 🚀 Docker Project Template for Data/ML Projects

## 📂 Folder Structure

Every new project starts like this:

```
myproject/
├── .dockerignore
├── Dockerfile
├── requirements.txt
├── requirements-dev.txt
└── main.py
```

(Optional: add `src/` and `notebooks/` as your project grows.)

```
src/
  test_claude.py
  data_pipeline.py
  agent_loop.py
```

In [ ]:
myproject/
├── .dockerignore          # excludes junk/secrets from images
├── Dockerfile             # recipe for building container
├── requirements.txt       # runtime dependencies (Claude, rich, etc.)
├── requirements-dev.txt   # developer dependencies (Jupyter, pytest, etc.)
├── .env                   # API keys (never committed, ignored by Git)
│
├── main.py                # minimal entrypoint (system check + hello Claude)
│
├── claude_chat.py         # Claude chat helper functions (multi-turn chat, pretty printing)
│
└── src/                   # your experiments & agent scripts
    ├── chat_loop.py       # interactive REPL with Claude
    ├── data_pipeline.py   # future data/ML experiments
    └── agent_loop.py      # agent orchestration logic



## 📄 1. `.dockerignore`

Keeps junk, secrets, and large data out of your images.


In [ ]:
# Python cache
__pycache__/
*.pyc
*.pyo
*.pyd

# Virtual environments
venv/
env/
.venv/

# Jupyter Notebook checkpoints
.ipynb_checkpoints/

# OS files
.DS_Store
Thumbs.db

# Git and version control
.git
.gitignore
.gitattributes

# Logs & debug
*.log
*.out
*.err

# Data & models (better to mount at runtime!)
data/
datasets/
*.csv
*.parquet
*.h5
*.pkl

# Large results
outputs/
results/
checkpoints/

# Secrets
.env
*.secret
*.key


## 📄 2. `requirements.txt`

**Runtime essentials** (only what your app needs to run):




In [ ]:
anthropic==0.34.0
rich==13.7.1
numpy==1.26.4
pandas==2.2.2
scikit-learn==1.5.1

## 📄 3. `requirements-dev.txt`

**Developer extras** (useful for notebooks, testing, formatting):


In [ ]:
jupyterlab==4.3.0
ipykernel==6.29.5
pytest==8.3.3
flake8==7.1.1
black==24.8.0

## 📄 4. `Dockerfile`

The full environment recipe:



In [ ]:
FROM python:3.11-slim

# Prevents pyc files, forces flush of stdout/stderr
ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1

# System deps
RUN apt-get update && apt-get install -y --no-install-recommends \
    build-essential \
    curl \
    git \
    wget \
    ca-certificates \
    && rm -rf /var/lib/apt/lists/*

# Workdir
WORKDIR /app

# Copy dependency files first (better cache)
COPY requirements.txt requirements-dev.txt ./

# Install runtime deps
RUN pip install --no-cache-dir -r requirements.txt

# Optional dev deps
ARG INSTALL_DEV=false
RUN if [ "$INSTALL_DEV" = "true" ] ; then pip install --no-cache-dir -r requirements-dev.txt ; fi

# Copy code
COPY . .

# Default run
CMD ["python", "main.py"]

## 📄 5. `main.py`

Your entrypoint for testing:

Great question — you actually have **two different use cases** here, and the best practice is to split them into two separate scripts so things stay organized:

---

### 1. ✅ `main.py`

This should stay **minimal**. Its job is to:

* Confirm your container runs correctly.
* Do a quick “hello world” check with Claude.

That way, when you run:

```bash
docker run --rm --env-file .env myproject:latest
```

it prints system info and sends a quick test message to Claude.


In [ ]:
import sys, platform, os
from claude_chat import chat_with_claude, reset_conversation

def main():
    name = os.getenv("NAME", "world")
    print(f"Hello, {name} 👋")
    print("Python:", platform.python_version())
    print("OS:", platform.platform())
    print("Args:", sys.argv[1:])

    reset_conversation()
    chat_with_claude("Hello Claude, can you confirm you’re working?")

if __name__ == "__main__":
    main()





### 2. 💬 `src/chat_loop.py`

This second block of code is an **interactive REPL** (a mini chat app). It doesn’t belong in `main.py` — better to put it in `src/chat_loop.py`:

```python
from claude_chat import chat_with_claude, reset_conversation

if __name__ == "__main__":
    reset_conversation()
    print("Claude Chat REPL (type 'exit' to quit)\n")
    while True:
        user_input = input("You: ")
        if user_input.lower() in {"exit", "quit"}:
            print("Goodbye 👋")
            break
        chat_with_claude(user_input)
```

Then you run it with:

```bash
docker run --rm -it --env-file .env myproject:latest python src/chat_loop.py
```

* `--rm` = auto-delete container after exit
* `-it` = interactive terminal
* `--env-file .env` = inject your Claude key
* `python src/chat_loop.py` = override the default `CMD` in your Dockerfile

---

## 🧠 Why split them?

* `main.py` = sanity check that your container works.
* `chat_loop.py` = real interactive tool for development.

Keeps things clean and avoids overloading your main entrypoint.



In [ ]:
from claude_chat import chat_with_claude, reset_conversation

if __name__ == "__main__":
    reset_conversation()
    print("Claude Chat REPL (type 'exit' to quit)\n")
    while True:
        user_input = input("You: ")
        if user_input.lower() in {"exit", "quit"}:
            print("Goodbye 👋")
            break
        chat_with_claude(user_input)




# 🚀 Docker Project Checklist (1-Page)

### 📂 Project Setup

```bash
mkdir myproject && cd myproject   # create project folder
nano Dockerfile                   # create Dockerfile
nano .dockerignore                 # create .dockerignore
nano requirements.txt              # runtime deps
nano requirements-dev.txt          # dev deps
nano main.py                       # entry script
```

### 📄 Typical Files

* **Dockerfile** → recipe for environment
* **.dockerignore** → exclude junk/secrets
* **requirements.txt** → runtime Python deps
* **requirements-dev.txt** → dev tooling
* **main.py** → entrypoint/test script

---

### 🛠️ Build & Run

```bash
docker build -t myproject:0.1 .              # build runtime image
docker build -t myproject:dev --build-arg INSTALL_DEV=true .   # build dev image

docker run --rm myproject:0.1                # run app
docker run --rm -e NAME=Micah myproject:0.1  # run with env var
docker run --rm --env-file .env myproject:0.1   # run with .env secrets
docker run -it myproject:0.1 bash            # get a bash shell inside container
docker run --rm -it -p 8888:8888 myproject:dev jupyter lab --ip=0.0.0.0 --allow-root  # Jupyter
```

---

# 💻 Basic Terminal Commands Cheat Sheet

### 📂 File & Folder Navigation

```bash
pwd               # print working directory
ls                # list files
ls -a             # list all files (including hidden)
cd foldername     # change directory
cd ..             # go up one level
```

### 📁 Create / Move / Delete

```bash
mkdir myfolder                # make a new folder
touch file.txt                # create empty file
mv oldname.txt newname.txt    # rename/move file
cp file.txt copy.txt          # copy file
rm file.txt                   # delete file
rm -r myfolder                # delete folder (recursive)
```

### 📄 Viewing Files

```bash
cat file.txt        # show entire file
less file.txt       # scroll through file (press q to quit)
head file.txt       # show first 10 lines
tail file.txt       # show last 10 lines
```

### 📝 Editing Files

```bash
nano file.txt       # edit file in nano (Ctrl+O save, Ctrl+X exit)
```

### ⚙️ System Info (inside container)

```bash
python --version    # check Python version
env                 # list environment variables
which python        # find where python is installed
```





# 🛠️ Usage Cheatsheet

### 1. Build image (runtime only)

```bash
docker build -t myproject:0.2 .
```

### 2. Build image with dev tools (Jupyter, pytest, etc.)

```bash
docker build -t myproject:dev --build-arg INSTALL_DEV=true .
```

### 3. Run container

```bash
docker run --rm --env-file .env myproject:0.2
```

### 4. Run with custom env var

```bash
docker run --rm -e NAME=Micah myproject:0.1
```

### 5. Run with `.env` file (API keys, secrets)

```bash
docker run --rm --env-file .env myproject:0.1
```

### 6. Run interactively (bash shell inside container)

```bash
docker run -it myproject:0.1 bash
```

### 7. Run Jupyter from dev image

```bash
docker run --rm -it -p 8888:8888 myproject:dev jupyter lab --ip=0.0.0.0 --allow-root
```

---

✅ With this template, you now have a **repeatable process**:

* Copy these files into a new folder.
* Edit `requirements.txt` for your project’s needs.
* Add your own Python scripts in `src/` or notebooks.
* Build and run.







## 🚀 Next Step: Using Claude (or OpenAI) Inside Your Container

Here’s the general flow you’ll follow:

1. Add the Claude (Anthropic) or OpenAI client library to `requirements.txt`.
   For example, to start with Claude:

   ```txt
   anthropic==0.34.0
   ```

   (for OpenAI, it’d be `openai==1.50.0` or similar)

2. Rebuild your container:

   ```bash
   docker build -t myproject:0.2 .
   ```

3. Keep your API keys in `.env`:

   ```env
   ANTHROPIC_API_KEY=claude-xxxx
   OPENAI_API_KEY=sk-xxxx
   ```

4. Run the container with secrets loaded:

   ```bash
   docker run --rm --env-file .env myproject:0.2
   ```

5. In `main.py`, you can now safely do:

   ```python
   import os
   from anthropic import Anthropic

   client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
   resp = client.messages.create(
       model="claude-3-5-sonnet-20240620",
       max_tokens=50,
       messages=[{"role": "user", "content": "Hello Claude!"}]
   )
   print(resp.content[0].text)
   ```




## Add .env file to Container

## 🔑 Step 1: Create your `.env` file (on your Mac, not in the container)

In your `myproject/` folder, make a file named `.env`:

```bash
nano .env
```

Paste something like this:

```env
ANTHROPIC_API_KEY=claude-xxxx
OPENAI_API_KEY=sk-xxxx
```

Save and exit.

⚠️ Remember:

* `.env` is in both `.dockerignore` and `.gitignore`.
* That means it stays local and private — not in your image, not on GitHub.

---

## 🔑 Step 2: Run container with `.env`

When you start your container, load the `.env` file:

```bash
docker run --rm --env-file .env myproject:0.2
```

* `--env-file .env` tells Docker to inject every variable from `.env`.
* Inside the container, those values become **environment variables**.

---

## 🔑 Step 3: Access variables in Python

In your code (`main.py` or any script), use `os.getenv`:

```python
import os
from anthropic import Anthropic

client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

resp = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=50,
    messages=[{"role": "user", "content": "Hello Claude!"}]
)

print(resp.content[0].text)
```

If you later add `OPENAI_API_KEY` to `.env`, you can grab it the same way:

```python
openai_key = os.getenv("OPENAI_API_KEY")
```

---

✅ This way:

* Your API keys stay **outside** the image.
* They’re injected at runtime when you run `docker run --env-file .env ...`.
* Your Python code just calls `os.getenv("ANTHROPIC_API_KEY")`.




## Claude Chat Script

In [2]:
script ='''\
import os, textwrap
from anthropic import Anthropic
from anthropic._exceptions import APIError
from rich.console import Console
from rich.markdown import Markdown

# API setup
anthropic_key = os.getenv("ANTHROPIC_API_KEY")
if not anthropic_key:
    raise RuntimeError("Missing ANTHROPIC_API_KEY environment variable.")

client = Anthropic(api_key=anthropic_key)
MODEL_NAME = "claude-3-5-sonnet-20240620"

# Conversation history
conversation = []

# Pretty printing setup
console = Console()

def smart_print_markdown(output: str, width: int = 100):
    """Wrap plain text, preserve fenced code blocks."""
    in_code = False
    para_buf = []

    def flush_paragraph():
        if para_buf:
            text = " ".join(para_buf)
            print(textwrap.fill(text, width=width, replace_whitespace=False))
            print()
            para_buf.clear()

    for line in output.splitlines():
        fence = line.strip().startswith("```")
        if fence:
            flush_paragraph()
            print(line)
            in_code = not in_code
            continue

        if in_code:
            print(line)
        else:
            if line.strip() == "":
                flush_paragraph()
            else:
                para_buf.append(line)

    flush_paragraph()

def chat_with_claude(
    prompt: str,
    system: str = "You are a helpful coding assistant.",
    render: str = "markdown",      # 'markdown' | 'wrapped' | 'none'
    return_text: bool = False,
    wrap_width: int = 100,
) -> str | None:
    """Send a prompt with conversation memory."""
    if not anthropic_key:
        raise RuntimeError("Missing ANTHROPIC_API_KEY.")

    conversation.append({"role": "user", "content": prompt})

    try:
        msg = client.messages.create(
            model=MODEL_NAME,
            max_tokens=1000,
            temperature=0.2,
            system=system,
            messages=conversation,
        )
        parts = [b.text for b in msg.content if getattr(b, "type", None) == "text"]
        output = "\n\n".join(parts).strip() or "(No text)"

        if render == "markdown":
            console.print(Markdown(output))
        elif render == "wrapped":
            smart_print_markdown(output, width=wrap_width)

        conversation.append({"role": "assistant", "content": output})
        return output if return_text else None

    except APIError as e:
        print("Anthropic API error:", e)
        raise

def reset_conversation():
    """Clear conversation history."""
    conversation.clear()

def last_reply() -> str | None:
    """Get last assistant message."""
    for m in reversed(conversation):
        if m["role"] == "assistant":
            return m["content"]
    return None

'''

# Write to file
with open("claude_chat.py", "w") as f:
    f.write(script)
